In [1]:
from sklearn.cluster import KMeans 
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import os
import plotly
import plotly.express as px
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# Data Getting

In [2]:
#read a file from this url "https://drive.google.com/file/d/1JNncMKuaXSwuQEp-S6wui4jiDcPGoCkU/view?usp=share_link" and save it as groups_data
groups_data = pd.read_csv("/Users/sebastianrobledo/Documents/Research/3BIO_analytics/info_grupos_merged_separado_instituciones.csv")

# Data describing

In [3]:
# Show me the column names of groups_data
groups_data.columns

Index(['codigo del grupo', 'nombre grupo', 'lider', 'avalado', 'estado',
       'clasificado en', 'link_grupos', 'link_perfiles', 'programa', 'area',
       'instituciones', 'fecha_creacion', 'lugar_creacion', 'gran_area',
       'anio'],
      dtype='object')

In [4]:
# count the number of rows and columns in groups_data
groups_data.shape

(13081, 15)

In [5]:
# Count the unique values in the column "codigo del grupo"
groups_data["codigo del grupo"].nunique()


5952

In [6]:
# create a dataframe with the unique values of "codigo del grupo" and other column with the number of times that value appears
groups_data["codigo del grupo"].value_counts() 

COL0016775    16
COL0111498    16
COL0078679    16
COL0147768    16
COL0132119    15
              ..
COL0189786     1
COL0187478     1
COL0195747     1
COL0105267     1
COL0211751     1
Name: codigo del grupo, Length: 5952, dtype: int64

In [7]:
# show me the type of data in each column of groups_data
groups_data.dtypes

codigo del grupo    object
nombre grupo        object
lider               object
avalado             object
estado              object
clasificado en      object
link_grupos         object
link_perfiles       object
programa            object
area                object
instituciones       object
fecha_creacion      object
lugar_creacion      object
gran_area           object
anio                 int64
dtype: object

In [8]:
# create a dataframe called estado with column "estado" and remove the string "Categoría " from the values in that column
estado = groups_data["estado"].str.replace("Categoría ","")

In [9]:
# Create a dataframe from groups_data filtered by the unique values in the column "codigo del grupo"
groups_data_filtered = groups_data.groupby("codigo del grupo").first()

In [10]:
# Show me the shape of groups_data_filtered
groups_data_filtered.shape

(5952, 14)

In [11]:
# Show me the column names of groups_data_filtered
groups_data_filtered.columns

Index(['nombre grupo', 'lider', 'avalado', 'estado', 'clasificado en',
       'link_grupos', 'link_perfiles', 'programa', 'area', 'instituciones',
       'fecha_creacion', 'lugar_creacion', 'gran_area', 'anio'],
      dtype='object')

In [12]:
# Create a dataframe called groups_filtered with the columns "nombre grupo", "estado", "instituciones", "fecha_creacion", "anio"
groups_filtered = groups_data_filtered[["nombre grupo", "estado", "instituciones", "fecha_creacion", "anio"]]

In [13]:
# Group the values in the column "estado" and count the number of times each value appears, save it as estado_count
estado_count = groups_filtered["estado"].value_counts()

In [14]:
# Change the values equal to "Categoría 00" to "Grupo Reconocido" in the column "estado" from groups_filtered
groups_filtered["estado"] = groups_filtered["estado"].replace("Categoría 00", "Grupo reconocido")


/var/folders/8y/j69blw9d67l72pygvntywxj00000gn/T/ipykernel_1726/4027708931.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  groups_filtered["estado"] = groups_filtered["estado"].replace("Categoría 00", "Grupo reconocido")


In [15]:
# Group the values in the column "estado" and count the number of times each value appears, save it as estado_count_1
estado_count_1 = groups_filtered["estado"].value_counts()

In [16]:
# Create a dataframe with the unique values of instituciones and other column with the number of times that value appears and save it as instituciones_count. Name the new column "count"
instituciones_count = groups_filtered["instituciones"].value_counts().reset_index(name="count")


In [17]:
# Create a dataframe with the matrix of the values in the column "instituciones" and the column "estado" and save it as estado_instituciones
estado_instituciones = pd.crosstab(groups_filtered["instituciones"], groups_filtered["estado"])


In [18]:
# Show me the shape of estado_instituciones
estado_instituciones.shape

(770, 5)

In [19]:
# Show the shape of instituciones_count
instituciones_count.shape

(770, 2)

In [20]:
# Merge the dataframes estado_instituciones and instituciones_count by the column "instituciones" from estado_instituciones and the column "index" from instituciones_count
estado_instituciones = estado_instituciones.merge(instituciones_count, left_index=True, right_on="index")

In [21]:
# add a column to groups_filtered called "years" with the difference between the current year and the year in the column "anio"
groups_filtered["years"] = 2022 - groups_filtered["anio"]

/var/folders/8y/j69blw9d67l72pygvntywxj00000gn/T/ipykernel_1726/869303622.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  groups_filtered["years"] = 2022 - groups_filtered["anio"]


In [22]:
# Change the name of the column "estado" to "category" in groups_filtered
groups_filtered = groups_filtered.rename(columns={"estado": "category"})

In [23]:
# Remove the string "Categoría " from the values in the column "category" from groups_filtered
groups_filtered["category"] = groups_filtered["category"].str.replace("Categoría ","")

In [24]:
# export the dataframe groups_filtered to a csv file called "groups_filtered.csv"
groups_filtered.to_csv("groups_filtered.csv")